In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import levitate
from importlib import reload

%load_ext snakeviz
%load_ext line_profiler

levitate.logger.setLevel(levitate.logging.WARNING)
levitate.logger.addHandler(levitate.logging.FileHandler('Optim.log', mode='w'))
levitate.logger.propagate = False

In [ ]:
transducer = levitate.models.CircularRing(effective_radius=2.9e-3, freq=40e3)
array = levitate.models.TransducerArray(transducer_model=transducer)

In [ ]:
loc = np.array([30e-3, 0, 0.2])
optim = levitate.optimization.Optimizer(array)
gorkov_1 = levitate.optimization.gorkov_laplacian(array=array, location=loc, weights=(1e-3, 1, 1, 1))
optim.add_objective(gorkov_1, 1)

In [ ]:
starting_phase = np.random.uniform(-np.pi, np.pi, array.num_transducers)
array.phases = starting_phase
array.amplitudes = 0.1*np.ones(array.num_transducers)

In [ ]:
optim.basinhopping = False
optim.variable_amplitudes = True
# %timeit optim.func_and_jac(array.phases)
# %snakeviz optim()
# %lprun -f optim.func_and_jac optim()
optim()
array.phases = np.mod(optim.phases + np.pi,2*np.pi)-np.pi
array.amplitudes = optim.amplitudes

In [ ]:
plot_loc = loc
p_scale = lambda p: 20*np.log10(np.abs(p)) + 94
p_range = (140, 160)

resolution = array.wavelength/15
xz_mesh = np.meshgrid(np.arange(-0.1, 0.1, resolution), np.arange(0, 0.3, resolution), indexing='ij')
pressures_xz = np.squeeze(array.calculate_pressure((xz_mesh[0], plot_loc[1]*np.ones_like(xz_mesh[0]), xz_mesh[1])))

fig = plt.figure()
plt.pcolormesh(xz_mesh[0], xz_mesh[1], p_scale(pressures_xz))
plt.xlabel('x')
plt.ylabel('z')
plt.clim(p_range)
plt.colorbar()
plt.plot(loc[0], loc[2], color='red', marker='o', markerfacecolor='red')
fig.show()

In [ ]:
fig = plt.figure()
plt.scatter(array.transducer_positions[:,0], array.transducer_positions[:,1], c=array.phases/np.pi, s=144, cmap='hsv')
plt.colorbar()
plt.title('Phase')
fig.show()

fig = plt.figure()
plt.scatter(array.transducer_positions[:,0], array.transducer_positions[:,1], c=array.amplitudes, s=144)
plt.colorbar()
plt.title('Amplitude')
fig.show()
plt.show()